# Segment Overlap Analysis

In [76]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(r'C:\Users\rafael_fagundes\OneDrive - Dell Technologies\Documents\DB\Liveramp.db')
df = pd.read_sql_query('SELECT * FROM TB_BILLING_DATA_NA', conn)
conn.close()


df.head()

,Org,BillingReportDate,OrgID,CoreSegment,CoreSegmentID,Partner,CoreSegmentImpressions,OrderCode,Provider,ProviderUUID,ProviderSegment,ProviderSegmentCode,DataProviderSegmentID,CPM,PriceModel,SalesforceBilled,Impressions,Cost,UsageDate,ProviderType
0,Dell NA,2020-03,4410,bargain shoppers_us-consumer_3p_acxiom_1x90,us2cpbpb2,MediaMath,367813,None,Acxiom,f19b7626-3732-4dcc-bac5-8d2c937dae9a,Bargain Shoppers,300730999,K-rUmWrc,1.0,CPM,Y,6712,6.71,2020-02,3rd
1,Dell NA,2020-03,4410,smart tech sb shopers_us-sb_3p_neustar_1x90,us92mfavz,MediaMath,460772,None,AdAdvisor Audiences by Neustar,2111c0af-fc3a-446f-ab07-63aa74fbde8e,Company Size: 1 employee,906697,JdXW-jSC,1.5,CPM,Y,87,0.13,2020-02,3rd
2,Dell NA,2020-03,4410,smart tech sb shopers_us-sb_3p_neustar_1x90,us92mfavz,MediaMath,460772,None,AdAdvisor Audiences by Neustar,2111c0af-fc3a-446f-ab07-63aa74fbde8e,Company Size: 1-4 employees,906695,JdXW-hOh,1.5,CPM,Y,590,0.89,2020-02,3rd
3,Dell NA,2020-03,4410,smart tech sb shopers_us-sb_3p_neustar_1x90,us92mfavz,MediaMath,460772,None,AdAdvisor Audiences by Neustar,2111c0af-fc3a-446f-ab07-63aa74fbde8e,Company Size: 20-100 employees,906696,JdXW-iNl,1.5,CPM,Y,119,0.18,2020-02,3rd
4,Dell NA,2020-03,4410,smart tech sb shopers_us-sb_3p_neustar_1x90,us92mfavz,MediaMath,460772,None,AdAdvisor Audiences by Neustar,2111c0af-fc3a-446f-ab07-63aa74fbde8e,Company Size: 5-19 employees,18392,KKO0_J3T,1.5,CPM,Y,467,0.70,2020-02,3rd


In [77]:
df['CoreSegment'] = df['CoreSegment'].str.replace('-','_')
df['CoreSegment'] = df['CoreSegment'].str.lower()

In [89]:
filter_str = '(mb |_csg|csg |_uw_|_sb_|_consumer_|_ce_|_isg_|_mb_|_csg_|_com_|_ce_|_isg_|_mb_|_csg_|_com_|_gaming_|_wafs_|_mpp_|_brand_|_uw_|_all_)'

df['BU'] = df['CoreSegment'].str.extract(filter_str)

df['BU'] = df['BU'].str.replace('_','')

df.to_csv(r'C:\Users\rafael_fagundes\OneDrive - Dell Technologies\Desktop\review.csv')

In [79]:
df = df[df['ProviderType'] == '3rd']

In [80]:
df_group = df.groupby(['CoreSegment','Provider','ProviderSegment'])['Impressions'].sum().reset_index()

df_group.head()

,CoreSegment,Provider,ProviderSegment,Impressions
0,100+ bdm_us_brand_3p_dun & bradstreet_1x90,D&B,Advertising,2421
1,100+ bdm_us_brand_3p_dun & bradstreet_1x90,D&B,Computing & IT,47766
2,100+ bdm_us_brand_3p_dun & bradstreet_1x90,D&B,Data Management,1669
3,100+ bdm_us_brand_3p_dun & bradstreet_1x90,D&B,Data Scientist / Analyst,3316
4,100+ bdm_us_brand_3p_dun & bradstreet_1x90,D&B,Database Administration & Design,883


In [81]:
df_group['ProviderSegment'].value_counts()

Operations                                               280
1                                                        270
Finance                                                  250
Information Technology                                   244
Finance Decision Makers                                  230
                                                        ... 
Online Interest in Digital Transformation Solutions        1
Online Interest in Internet Of Things (IOT) Solutions      1
Online Interest in Laptops                                 1
Online Interest in Networking Equipment & Solutions        1
Millennials                                                1
Name: ProviderSegment, Length: 1349, dtype: int64

In [82]:
overlap_df = df.groupby('ProviderSegment').agg({'CoreSegment': pd.Series.nunique, 'Provider': pd.Series.nunique})

overlap_df = overlap_df[(overlap_df['CoreSegment'] > 1) | (overlap_df['Provider'] > 1)]

overlap_df.sort_values(by='CoreSegment', ascending=False)

,CoreSegment,Provider
ProviderSegment,,
1,270,1
Operations,156,6
IT Decision Makers,149,4
Finance Decision Makers,146,5
Finance,131,8
...,...,...
Paper,2,1
Parents of Children (Ages 6-10),2,1
"$1,000,000-$5,000,000",2,1


In [83]:
overlap_df.to_csv(r'C:\Users\rafael_fagundes\OneDrive - Dell Technologies\Desktop\overlap_df.csv')

In [84]:
df_group[df_group['ProviderSegment'] == 'Operations'].to_csv(r'C:\Users\rafael_fagundes\OneDrive - Dell Technologies\Desktop\test.csv')